In [9]:
import pandas as pd
import numpy as np

In [10]:
wages = pd.read_csv("wage data")
marijuana = pd.read_csv("marijuana years df")
marijuana = marijuana.iloc[0:51]
marijuana = marijuana.set_index(["State"])
marijuana = marijuana[["Year legalized (Rec)", "Year legalized (Med)"]]
marijuana = marijuana.rename(columns={"Year legalized (Rec)" : "rec", "Year legalized (Med)" : "med"})
marijuana.index.names = ["state"]



In [11]:
# Create a list of all possible state-year combinations from 2001 to 2022
state_list = marijuana.index.unique()
year_range = list(map(str, range(2001, 2023)))  # Convert years to strings
index = pd.MultiIndex.from_product([state_list, year_range], names=["State", "Year"])

# Create an empty DataFrame with the multi-index
pivoted_df = pd.DataFrame(index=index)



In [12]:
pivoted_df.insert(loc=0,column="rec_legal",value=np.nan)
pivoted_df.insert(loc=1,column="med_legal",value=np.nan)

In [13]:
#would be more efficient to this the other way around, but I realzied that after starting and on a 10^4 dataset running once I can't be bothered to rewrite
for state, year in pivoted_df.index:
    for manner in ["rec", "med"]:
        value = marijuana[manner].loc[state]
        if isinstance(value, (float, int)) and not pd.isnull(value) and value <= int(year):
            pivoted_df.loc[(state, year), f"{manner}_legal"] = True
        else: 
            pivoted_df.loc[(state, year), f"{manner}_legal"] = False


FrozenList(['State', 'Year'])

In [15]:
wages

,Unnamed: 0,state,year,all_occ_h_median,drivers_h_median
0,0,Alabama,2001,11.02,12.01
1,1,Alaska,2001,16.12,16.30
2,2,Arizona,2001,12.33,12.84
3,3,Arkansas,2001,10.47,12.28
4,4,California,2001,14.31,14.01
...,...,...,...,...,...
1117,1117,Virginia,2022,23.22,20.72
1118,1118,Washington,2022,27.08,25.81
1119,1119,West Virginia,2022,18.16,18.85
1120,1120,Wisconsin,2022,21.95,21.90


In [17]:
wages_reset = wages.reset_index()
pivoted_df_reset = pivoted_df.reset_index()
wages_reset = wages_reset.sort_values(by=(["year", "state"]), ignore_index=True)
pivoted_df_reset = pivoted_df_reset.rename(columns = {"Year" : "year", "State" : "state"})
pivoted_df_reset = pivoted_df_reset.sort_values(by=(["year", "state"]), ignore_index=True)

In [21]:
out_df = pd.concat(objs=[wages_reset, pivoted_df_reset], axis=1)
out_df = out_df.loc[:, ~out_df.columns.duplicated()]
out_df = out_df[["state", "year", "all_occ_h_median", "drivers_h_median", "rec_legal", "med_legal"]]

In [23]:
out_df.to_csv("data")